# SIMULATION OF NLOS-VLC SYSTEM USING THE ROLLING SHUTTER MECHANISM

### Import the transmitter, surface and camera objects, as well as numpy library.

In [ ]:
#Import module Transmitter
from transmitter import Transmitter as Transmitter

from surface import Surface as Surface

from camera import Camera as Camera

from rollingshutter import RollingShutter as RS

import numpy as np


### The trasmitter is defined as a Trasmitter type object from nine input parameters


In [ ]:
transmitter = Transmitter(
        "Led1",
        position=[0, 0, 2],
        normal=[0, 1, -1],
        mlambert=1,
        wavelengths=[620, 530, 475],
        fwhm=[20, 30, 20],
        modulation='ieee16',
        frequency=2000,
        no_symbols=1000,
        luminous_flux=1000
    )
# transmitter.plot_spd_normalized()
# transmitter.plot_spd_at_1lm()
# transmitter.plot_led_pattern()
print(transmitter)

### The surface is defined from eight imputs parametes

In [ ]:
surface = Surface(
    name="surfacePlaster",
    position=[0, 2, 0],
    reflectance='plaster',
    vertex1=[-1/10, 0, -1/10],
    vertex2=[1/10, 0, -1/10],
    vertex3=[1/10, 0, 1/10],
    vertex4=[-1/10, 0, 1/10]
    )
# surface.plot_reflectance()
surface._group_vertices()
print(surface)

### The camera is defined from a set of 13 input parameters

In [ ]:
PIXEL_SIZE = 1.12e-6
MX = 1/PIXEL_SIZE  # number of pixels per unit distance in image coordinates in x direction
MY = 1/PIXEL_SIZE  # number of pixels per unit distance in image coordinates in y direction
FOCAL_LENGTH = 3e-3  # focal length
THETA_X = np.pi / 2.0  # roll angle
THETA_Y = np.pi  # pitch angle
THETA_Z = np.pi  # yaw angle
C = np.array([0, 1, 0])  # camera centre
IMAGE_WIDTH = 1200
IMAGE_HEIGTH = 800
PX= IMAGE_WIDTH/(2*MX)  # principal point x-coordinate
PY= IMAGE_HEIGTH/(2*MY)  # principal point y-coordinate

camera = Camera(
    name="camera1",
    focal_length=FOCAL_LENGTH,
    pixel_size=1/MX,
    px=PX,
    py=PY,
    mx=MX,
    my=MY,
    theta_x=THETA_X,
    theta_y=THETA_Y,
    theta_z=THETA_Z,
    centre=C,
    image_height=IMAGE_HEIGTH,
    image_width=IMAGE_WIDTH,    
    surface=surface,
    transmitter=transmitter,
    sensor='SonyStarvisBSI'
    )
camera.plot_responsivity()
camera.plot_image_intensity()



### The Rolling Shutter module is added.

In [ ]:
# Library to plot 
import matplotlib.pyplot as plt

camera.idark = 1.9e-23

rollingshutter = RS(
    name="rs",
    t_exposure=85e-6,
    t_rowdelay=18.904e-6,
    t_start=0,
    iso=200,
    adc_resolution=8,
    gain_pixel=13e20,
    temperature=298,
    transmitter=transmitter,
    camera=camera
)

rollingshutter.plot_color_image()
rollingshutter.add_blur(size=11, center=5.5, sigma=10.5)
rollingshutter.plot_blurred_image()


